## LSTM for IMBD Sentiment Analysis

In this notebook we will ...

In [14]:
import torch
from torchtext import data
from torchtext import datasets
import random

## Download and Read Data

You should install `spacy` using `pip install spacy`.

In [15]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz:  18%|█▊        | 15.3M/84.1M [04:00<09:36, 119kB/s] 

In [5]:
print(vars(train_data.examples[0]))

IndexError: list index out of range

In [ ]:
SEED = 1230245
train_data, valid_data = train_data.split(random_state = random.seed(SEED))


## Build Vocabulary

In [ ]:

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

## Explore Features

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

In [ ]:
print(TEXT.vocab.itos[:10])

In [ ]:
print(LABEL.vocab.stoi)

## Peprare for training

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)